# JupyterLite で学ぶ Leaflet / folium 入門チュートリアル

このノートブックは、**JupyterLite（ブラウザだけで動く Jupyter 環境）** 上で、
JavaScript の地図ライブラリ **Leaflet** を Python から扱うためのラッパーライブラリ **folium** を使い、
インタラクティブな地図を表示・操作する方法を学ぶためのチュートリアルです。

主な内容：
- folium / Leaflet の概要
- 基本の地図表示
- マーカーとポップアップ
- 複数地点の可視化（ループ・DataFrame）
- タイルレイヤの変更
- GeoJSON の表示
- ミニ演習

※ JupyterLite 上で folium を利用するために、純 Python パッケージとして `micropip` からインストールしてみます。

## 0. 環境準備（JupyterLite 用）

このノートブックは **JupyterLite（Pyodide）** を想定しています。

まず、folium をインストールし、必要なライブラリをインポートします。
※ ネットワーク制限や環境によってはインストールに失敗することがあります。

In [1]:
# folium は純 Python パッケージのため、micropip でインストールを試みます。
try:
    import folium
except ImportError:
    try:
        import micropip
        await micropip.install("folium")
        import folium
    except Exception as e:
        print("folium のインストールに失敗しました:", e)
        print("この環境で folium が利用できない場合があります。")

import pandas as pd
import numpy as np

print("ライブラリの読み込みが完了しました（エラーが出ていなければ OK）。")

ライブラリの読み込みが完了しました（エラーが出ていなければ OK）。


## 1. folium と Leaflet の概要

**Leaflet** は、ブラウザ上で動作する軽量な JavaScript 製の地図ライブラリです。
OpenStreetMap などのタイル地図をベースに、マーカーやポリゴン・ポリラインなどを簡単に描画できます。

**folium** は、この Leaflet を Python から扱うためのラッパーライブラリで、
Jupyter Notebook / JupyterLite の中でインタラクティブな地図を表示できます。

folium の基本的な使い方は：
1. `folium.Map` オブジェクトを作る
2. `folium.Marker` などで要素を追加
3. Notebook 上でオブジェクトを表示
という流れです。

## 2. 基本の地図表示

まずは、もっとも基本的な「地図を 1 枚表示する」ことから始めます。

ここでは例として、名古屋市瑞穂区滝子町にある名古屋市立大学経済学部付近の緯度経度（35.13, 136.93 あたり）を中心に表示してみます。

### 2.1 シンプルな地図

In [2]:
# 名古屋市立大学経済学部（滝子キャンパス）付近の座標（緯度・経度）
center = [35.1330, 136.9264]

m = folium.Map(location=center, zoom_start=12)

# Notebook 上で地図オブジェクトを評価すると、HTML として表示されます
m

### 2章 練習問題

1. 自分の住んでいる地域、または行ってみたい都市の緯度・経度を調べて、その地点を中心とした地図を表示しなさい。
2. `zoom_start` の値を 5, 10, 15 などに変えて、ズームレベルの違いを体験しなさい。
3. 世界地図レベル（例：`location=[0, 0], zoom_start=2`）で表示した地図も試し、日本との位置関係を確認しなさい。

## 3. マーカーとポップアップ

地図に「印（ピン）」を立てるには、`folium.Marker` を使います。
クリックしたときに表示されるポップアップ文字列も簡単に設定できます。

### 3.1 名古屋市立大学経済学部にマーカーを立てる例

In [3]:
# 名古屋市立大学経済学部
ncu_economics = [35.1330, 136.9264]

m = folium.Map(location=center, zoom_start=14)
folium.Marker(
    location=ncu_economics,
    popup="名古屋市立大学経済学部",
).add_to(m)

m

### 3.2 複数のマーカーを追加する

複数の地点をリストで用意して、for ループでまとめてマーカーを追加してみます。

In [4]:
points = [
    {"name": "名古屋市立大学経済学部", "lat": 35.1330, "lon": 136.9264},
    {"name": "金山駅", "lat": 35.1438, "lon": 136.9005},
    {"name": "熱田神宮", "lat": 35.1281, "lon": 136.9077},
]

m = folium.Map(location=center, zoom_start=12)

for p in points:
    folium.Marker(
        location=[p["lat"], p["lon"]],
        popup=p["name"],
    ).add_to(m)

m

### 3章 練習問題

1. 自分の好きなスポット（3〜5か所）の名称と緯度経度を調べ、リストや DataFrame を使ってまとめてマーカーを表示しなさい。
2. `popup` に日本語の説明文をもう少し長めに入れてみて、どのように表示されるか確認しなさい。
3. `tooltip` 引数を使うと、マーカーにマウスを乗せたときに表示される短いテキストを指定できます。`tooltip` を試してみなさい。

## 4. pandas DataFrame からマーカーを作る

地点情報が DataFrame として手元にある場合、行をループしながらマーカーを追加できます。
ここでは、簡単な飲食店リストを DataFrame で作り、地図に可視化してみます。

### 4.1 DataFrame を作成してマッピング

In [5]:
df_spots = pd.DataFrame({
    "name": ["カフェA", "レストランB", "ベーカリーC"],
    "lat": [35.130, 35.135, 35.125],
    "lon": [136.920, 136.930, 136.928],
    "category": ["cafe", "restaurant", "bakery"],
})

df_spots

,name,lat,lon,category
0,カフェA,35.130,136.920,cafe
1,レストランB,35.135,136.930,restaurant
2,ベーカリーC,35.125,136.928,bakery


In [6]:
m = folium.Map(location=center, zoom_start=13)

for _, row in df_spots.iterrows():
    folium.Marker(
        location=[row["lat"], row["lon"]],
        popup=f'{row["name"]} ({row["category"]})',
    ).add_to(m)

m

### 4章 練習問題

1. 独自の DataFrame（例：コンビニ、スーパー、病院など）を作成し、カテゴリ情報も含めてマーカーに表示しなさい。
2. カテゴリごとにマーカーの色を変えたい場合、どのようなロジックで `folium.Icon(color=...)` を指定すればよいか考えなさい。
3. 緯度・経度だけでなく、`url` 列を持たせ、ポップアップ内にリンク（HTML）を埋め込むこともできます。HTML 文字列を popup に与える方法を調べて試してみなさい。

## 5. タイルレイヤ（地図のデザイン）を変える

folium では、OpenStreetMap のほかにもさまざまなタイルレイヤを利用できます。
`folium.Map` の `tiles` 引数で指定します。

例：
- `"OpenStreetMap"`
- `"CartoDB positron"`
- `"CartoDB dark_matter"`
- `"CartoDB positron"` など

### 5.1 タイルを変えてみる例

In [7]:
tiles_list = ["OpenStreetMap", "CartoDB positron", "CartoDB dark_matter"]

maps = []
for t in tiles_list:
    m = folium.Map(location=center, zoom_start=12, tiles=t)
    maps.append(m)

# 最初のマップを表示（必要に応じて maps[1], maps[2] なども評価してみてください）
maps[0]

### 5章 練習問題

1. `tiles` に `"CartoDB positron"` を指定して地図を表示し、標準との見た目の違いを確認しなさい。
2. 教育・プレゼン用途で使うとき、どのようなタイルデザインが適切か、自分の使い方を想像して考えを述べなさい。
3. 背景地図の著作権表示（クレジット）は通常どう扱われているか、folium / Leaflet の表示を見て確認しなさい。

## 6. GeoJSON レイヤの表示

**GeoJSON** は、地理情報（ポイント・ライン・ポリゴンなど）を表現するための JSON 形式です。
folium では、`folium.GeoJson` を使って GeoJSON データを地図に重ねて表示できます。

ここでは、簡単なポリゴン（四角形）の GeoJSON をコード内に直接書いて表示してみます。

### 6.1 シンプルなポリゴン GeoJSON の例

In [8]:
# 非常に簡略化された四角形ポリゴン（名古屋市立大学経済学部付近のどこか）
simple_geojson = {
    "type": "FeatureCollection",
    "features": [
        {
            "type": "Feature",
            "properties": {"name": "サンプルエリア"},
            "geometry": {
                "type": "Polygon",
                "coordinates": [[
                    [136.920, 35.130],
                    [136.933, 35.130],
                    [136.933, 35.125],
                    [136.920, 35.125],
                    [136.920, 35.130]
                ]]
            }
        }
    ]
}

m = folium.Map(location=center, zoom_start=14)
folium.GeoJson(simple_geojson, name="geojson").add_to(m)
folium.LayerControl().add_to(m)

m

### 6章 練習問題

1. 自分で簡単なポリゴン（四角形や三角形）GeoJSON を作成し、色を変えて表示しなさい（`style_function` を使うと色指定ができます）。
2. 市区町村の境界 GeoJSON ファイルを入手した場合、JupyterLite にアップロードして `folium.GeoJson` で表示する手順を文章で説明しなさい。
3. ポリゴンにマウスを乗せたときに名前を表示するなど、インタラクティブな効果を付けたい場合、どのような拡張が考えられるか調べてみなさい。

## 7. 総合ミニ演習：店舗マップ / 観光マップを作る

最後に、これまで学んだ内容を組み合わせて、
自分だけの「店舗マップ」や「観光マップ」を作ってみましょう。

### 演習 1：自分の生活圏マップ

1. 自宅、よく行くスーパー、コンビニ、駅、カフェなど 5〜10 箇所の地点情報（名称・緯度・経度）を DataFrame やリストで作成しなさい。
2. folium を使って、これらの地点にマーカーを立てた地図を作成しなさい。
3. `popup` や `tooltip` に、その場所に対する自分のコメント（例：安い・静か・お気に入りなど）を加えなさい。
4. コードと地図のスクリーンショットをレポートなどに載せることを想定し、説明文をまとめてみなさい。

### 演習 2：観光案内マップ

1. ある都市（例：名古屋、京都、大阪など）を 1 つ選び、主要な観光スポットや駅、ホテルなど 5〜10 箇所をピックアップしなさい。
2. 観光スポットごとにカテゴリ（寺社仏閣、博物館、グルメ、自然など）を設定し、カテゴリ別にマーカーの色やアイコンを変えることを検討しなさい。
3. 地図上でスポットを眺めながら、「どの順番で回ると効率が良さそうか」「どのエリアにスポットが集中しているか」などを考察し、文章にまとめなさい。

---

これで、JupyterLite 上での **Leaflet / folium 入門チュートリアル** は終了です。

folium は、pandas や statsmodels と同様に、
データ分析の結果を **地図上に可視化するための強力なツール** です。

ぜひ、実際のデータ（例：調査地点、店舗売上、交通量、センサデータなど）にも応用してみてください。